In [0]:
import json
from includes.UpdateDashboardParams import update_parameters
import os

In [0]:
dbutils.widgets.text('catalog_name', defaultValue='')
dbutils.widgets.text('schema_name', defaultValue='')
dbutils.widgets.text('skip_parameter_update', defaultValue='False')
dbutils.widgets.text('dashboard_id', defaultValue='')
dbutils.widgets.text('deployed', defaultValue='false')

catalog_name = dbutils.widgets.get('catalog_name')
schema_name = dbutils.widgets.get('schema_name')
skip_update = dbutils.widgets.get('skip_parameter_update').lower() == 'true'
drillthrough_dash_id = dbutils.widgets.get('dashboard_id')
workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
deployed = dbutils.widgets.get('deployed').lower() == 'true'

assert catalog_name and schema_name, 'Catalog name and schema name inputs must be provided'

if skip_update:
  dbutils.notebook.exit('Skipping dashboard parameter update')

In [0]:
# Note if you are developing you want to use ./dashboards/. The ../../resources are the dashboards created during bundle deployment
if deployed:
  dashboard_root_path = '../../resources/'
else:
  dashboard_root_path = './dashboards/'
all_dashboards = [f for f in os.listdir(dashboard_root_path) if f.endswith('.lvdash.json')]

for dashboard in all_dashboards:
  target_file = os.path.join(dashboard_root_path, dashboard)
  print(f"Attempting to update: {target_file}")
  with open(target_file, 'r') as f:
    dashboard_json = json.load(f)
      # Write updated JSON
  updated_json = update_parameters(dashboard_json, catalog_name, schema_name, workspace_url, drillthrough_dash_id)
  
  with open(target_file, 'w') as f:
      json.dump(updated_json, f, indent=2)
  print(f'Updated dashboard: {target_file} with catalog = {catalog_name}, schema = {schema_name}, workspace_url = {workspace_url}, drillthrough dashboard_id = {drillthrough_dash_id}')


